<a href="https://colab.research.google.com/github/dannynacker/strobe_entrainment_periodicity_MSc/blob/main/aperiodic_generation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Current fixes to add:

The following are a series of scripts used to generate formatted .txt files for communication with another in-house stroboscope for the SCCS. Of these, #5 is the most powerful, with the capacity to specify multiple steps, the periodicity of the waveform, their durations, start and end effective frequency (in case an aperiodic waveform is chosen), start and end duty cycle, start and end luminance, a square- or sine- wave-modulated signal, and which of 4 LEDs are to be either ON or OFF per each oscillator (4 total). Recent adjustments include proper formatting of TIM to remove DAY stamps, as well as removal of redundant STP durations per oscillator within each row.

1. This version treats LED mapping specific to oscillator, such that the four STPs always go 1 0 0 0, 0 1 0 0, etc; which, after looking at the Session Manager more closely, is likely incorrect, but this version is still being retained for future edits should the LED-specific mapping go awry.

In [ ]:
import numpy as np

def poisson_flash_times(frequency, duration, duty_cycle, min_interval=0.025, max_interval_factor=3.0):
    """Generates Poisson-distributed flash onset times ensuring correct flash count and duty cycle adjustments."""
    cycle_length = 1.0 / frequency
    expected_flashes = int(duration * frequency)
    times = []
    time = 0
    while len(times) < expected_flashes and time < duration:
        interval = np.random.exponential(cycle_length * 0.8)
        interval = min(interval, cycle_length * max_interval_factor)
        interval = max(interval, min_interval)
        on_duration = interval * (duty_cycle / 100)
        time += interval
        if time < duration:
            times.append((time, on_duration))
    return times

def calculate_effective_frequency(flash_times, duration):
    """Calculate effective frequency from generated flash times."""
    return len(flash_times) / duration

def optimize_initial_frequency(target_Fe, duration, duty_cycle, tolerance=0.01, max_iterations=10000, max_freq=200.0):
    """Finds the optimal initial frequency that produces the desired effective frequency."""
    F = min(target_Fe * 1.2, max_freq)  # Start slightly higher
    iteration = 0
    while iteration < max_iterations:
        flash_times = poisson_flash_times(F, duration, duty_cycle)
        Fe = calculate_effective_frequency(flash_times, duration)
        if abs(Fe - target_Fe) <= tolerance:
            return min(F, max_freq), Fe, flash_times
        F = min(max(0.1, F + 0.15 if Fe < target_Fe else F - 0.15), max_freq)
        iteration += 1
    print(f"Warning: Max iterations reached for effective frequency {target_Fe} Hz.")
    return F, Fe, flash_times

def format_stp(time, frequency, duty_cycle, brightness, led_assignments):
    """
    Formats a single STP line with all four oscillator parameter blocks concatenated.
    Each block: step duration, wave type, start freq, end freq, start duty, end duty, LED config (4 values), start intensity, end intensity.
    The led_assignments parameter should be a list of four lists, one per oscillator.
    """
    # Build each oscillator's parameter block:
    blocks = []
    for osc in range(4):
        led_str = ",".join(map(str, led_assignments[osc]))
        # Here we use the same frequency, duty, and brightness for both start and end values.
        block = f"{time:.3f},1,{frequency:.2f},{frequency:.2f},{duty_cycle},{duty_cycle},{led_str},{brightness},{brightness}"
        blocks.append(block)
    # Combine all four blocks into one STP line (separated by commas, no extra 'STP' tokens in between)
    return f'STP"{",".join(blocks)}"'

def generate_strobe_sequence():
    """Generates a strobe sequence file with proper STP formatting for all four oscillators in one line per step."""
    stimulation = input("Choose stimulation type (Periodic/Aperiodic): ").strip()
    duration = float(input("Enter total sequence duration (seconds): "))
    start_Fe = float(input("Enter start effective frequency (Hz): "))
    end_Fe = float(input("Enter end effective frequency (Hz): "))
    start_l = int(input("Enter start luminance (0-100): "))
    end_l = int(input("Enter end luminance (0-100): "))
    start_d = int(input("Enter start duty cycle (1-99): "))
    end_d = int(input("Enter end duty cycle (1-99): "))
    wave_type = input("Enter wave type (Square/Sine): ").strip()
    # We now assume the default LED assignment as per the creator's info:
    # Osc1: 1,0,0,0; Osc2: 0,1,0,0; Osc3: 0,0,1,0; Osc4: 0,0,0,1.
    led_config = [[1,0,0,0],
                  [0,1,0,0],
                  [0,0,1,0],
                  [0,0,0,1]]

    # Header lines
    output = [f'TIM"00:00:{int(duration // 60):02}:{duration % 60:04.1f}"', f'DUR"{duration:.1f}"']

    # Determine number of steps (simple approximation)
    num_steps = int(duration * (start_Fe + end_Fe) / 2)
    min_interval = 0.025

    # Generate flash times based on stimulation type
    if stimulation == "Aperiodic":
        F, Fe, flash_times = optimize_initial_frequency(start_Fe, duration, start_d)
    else:
        flash_times = [(i * (1.0 / start_Fe), (start_d / 100) * (1.0 / start_Fe)) for i in range(num_steps)]
        Fe = start_Fe

    previous_onset = 0
    step_lines = []

    # For each flash time, generate one STP line (which includes the parameters for all 4 oscillators)
    for onset, on_duration in flash_times:
        step_duration = max(onset - previous_onset, min_interval)
        previous_onset = onset + on_duration

        # For simplicity, use start values for the first half and end values for the second half:
        step_freq = start_Fe if onset < duration / 2 else end_Fe
        step_duty = start_d if onset < duration / 2 else end_d
        step_luminance = start_l if onset < duration / 2 else end_l

        stp_line = format_stp(step_duration, step_freq, step_duty, step_luminance, led_config)
        step_lines.append(stp_line)

    output.extend(step_lines)

    file_name = "strobe_periodic.txt" if stimulation == "Periodic" else "strobe_aperiodic.txt"
    with open(file_name, "w") as f:
        f.write("\n".join(output))

    print(f"Generated {file_name}.")

# Run the generator
generate_strobe_sequence()

Choose stimulation type (Periodic/Aperiodic): Periodic
Enter total sequence duration (seconds): 10
Enter start effective frequency (Hz): 10
Enter end effective frequency (Hz): 10
Enter start luminance (0-100): 50
Enter end luminance (0-100): 50
Enter start duty cycle (1-99): 50
Enter end duty cycle (1-99): 50
Enter wave type (Square/Sine): Square
Generated strobe_periodic.txt.


2. This version allows for specific LED set mapping per each oscillator, adding an additional 16 questions to the user input prompt. I think this is more correct, compared to the initial fixed single-LED-per-oscillator function above.

In [4]:
import numpy as np

def poisson_flash_times(frequency, duration, duty_cycle, min_interval=0.025, max_interval_factor=3.0):
    cycle_length = 1.0 / frequency
    expected_flashes = int(duration * frequency)
    times = []
    time = 0
    while len(times) < expected_flashes and time < duration:
        interval = np.random.exponential(cycle_length * 0.8)
        interval = min(interval, cycle_length * max_interval_factor)
        interval = max(interval, min_interval)
        on_duration = interval * (duty_cycle / 100)
        time += interval
        if time < duration:
            times.append((time, on_duration))
    return times

def calculate_effective_frequency(flash_times, duration):
    return len(flash_times) / duration

def optimize_initial_frequency(target_Fe, duration, duty_cycle, tolerance=0.01, max_iterations=10000, max_freq=200.0):
    F = min(target_Fe * 1.2, max_freq)
    iteration = 0
    while iteration < max_iterations:
        flash_times = poisson_flash_times(F, duration, duty_cycle)
        Fe = calculate_effective_frequency(flash_times, duration)
        if abs(Fe - target_Fe) <= tolerance:
            return min(F, max_freq), Fe, flash_times
        F = min(max(0.1, F + 0.15 if Fe < target_Fe else F - 0.15), max_freq)
        iteration += 1
    print(f"Warning: Max iterations reached for effective frequency {target_Fe} Hz.")
    return F, Fe, flash_times

def format_stp(time, frequency, duty_cycle, brightness, led_assignments):
    blocks = []
    for osc in range(4):
        led_str = ",".join(map(str, led_assignments[osc]))
        block = f"{time:.3f},1,{frequency:.2f},{frequency:.2f},{duty_cycle},{duty_cycle},{led_str},{brightness},{brightness}"
        blocks.append(block)
    return f'STP"{",".join(blocks)}"'

def get_led_assignments():
    led_config = []
    for osc in range(4):
        osc_leds = []
        print(f"Enter LED assignments for Oscillator {osc+1} (0 [off] or 1 [on]):")
        for led in range(4):
            value = int(input(f"  LED {led+1}: "))
            while value not in [0, 1]:
                print("  Please enter 0 or 1.")
                value = int(input(f"  LED {led+1}: "))
            osc_leds.append(value)
        led_config.append(osc_leds)
    return led_config

def generate_strobe_sequence():
    stimulation = input("Choose stimulation type (Periodic/Aperiodic): ").strip()
    duration = float(input("Enter total sequence duration (seconds): "))
    start_Fe = float(input("Enter start effective frequency (Hz): "))
    end_Fe = float(input("Enter end effective frequency (Hz): "))
    start_l = int(input("Enter start luminance (0-100): "))
    end_l = int(input("Enter end luminance (0-100): "))
    start_d = int(input("Enter start duty cycle (1-99): "))
    end_d = int(input("Enter end duty cycle (1-99): "))
    wave_type = input("Enter wave type (Square/Sine): ").strip()

    led_config = get_led_assignments()

    output = [f'TIM"00:00:{int(duration // 60):02}:{duration % 60:04.1f}"', f'DUR"{duration:.1f}"']

    num_steps = int(duration * (start_Fe + end_Fe) / 2)
    min_interval = 0.025

    if stimulation == "Aperiodic":
        F, Fe, flash_times = optimize_initial_frequency(start_Fe, duration, start_d)
    else:
        flash_times = [(i * (1.0 / start_Fe), (start_d / 100) * (1.0 / start_Fe)) for i in range(num_steps)]
        Fe = start_Fe

    previous_onset = 0
    step_lines = []

    for onset, on_duration in flash_times:
        step_duration = max(onset - previous_onset, min_interval)
        previous_onset = onset + on_duration

        step_freq = start_Fe if onset < duration / 2 else end_Fe
        step_duty = start_d if onset < duration / 2 else end_d
        step_luminance = start_l if onset < duration / 2 else end_l

        stp_line = format_stp(step_duration, step_freq, step_duty, step_luminance, led_config)
        step_lines.append(stp_line)

    output.extend(step_lines)

    file_name = "strobe_periodic.txt" if stimulation == "Periodic" else "strobe_aperiodic.txt"
    with open(file_name, "w") as f:
        f.write("\n".join(output))

    print(f"Generated {file_name}.")

generate_strobe_sequence()

Choose stimulation type (Periodic/Aperiodic): Periodic
Enter total sequence duration (seconds): 10
Enter start effective frequency (Hz): 10
Enter end effective frequency (Hz): 10
Enter start luminance (0-100): 50
Enter end luminance (0-100): 50
Enter start duty cycle (1-99): 50
Enter end duty cycle (1-99): 50
Enter wave type (Square/Sine): Square
Enter LED assignments for Oscillator 1 (0 [off] or 1 [on]):
  LED 1: 1
  LED 2: 1
  LED 3: 1
  LED 4: 1
Enter LED assignments for Oscillator 2 (0 [off] or 1 [on]):
  LED 1: 1
  LED 2: 1
  LED 3: 1
  LED 4: 1
Enter LED assignments for Oscillator 3 (0 [off] or 1 [on]):
  LED 1: 1
  LED 2: 1
  LED 3: 1
  LED 4: 1
Enter LED assignments for Oscillator 4 (0 [off] or 1 [on]):
  LED 1: 1
  LED 2: 1
  LED 3: 1
  LED 4: 1
Generated strobe_periodic.txt.


3. The following is an adapation of the above [with specific LED/oscillator mapping], while allowing the user to specify distinct steps at which these changes should occur; that is, instead of having to generate multiple transitions using the scripts above and manually appending them, the number of steps and their durations are specified via user input.

In [ ]:
import numpy as np

def poisson_flash_times(frequency, duration, duty_cycle, min_interval=0.025, max_interval_factor=3.0):
    cycle_length = 1.0 / frequency
    expected_flashes = int(duration * frequency)
    times = []
    time = 0
    while len(times) < expected_flashes and time < duration:
        interval = np.random.exponential(cycle_length * 0.8)
        interval = min(interval, cycle_length * max_interval_factor)
        interval = max(interval, min_interval)
        on_duration = interval * (duty_cycle / 100)
        time += interval
        if time < duration:
            times.append((time, on_duration))
    return times

def calculate_effective_frequency(flash_times, duration):
    return len(flash_times) / duration

def optimize_initial_frequency(target_Fe, duration, duty_cycle, tolerance=0.01, max_iterations=10000, max_freq=200.0):
    F = min(target_Fe * 1.2, max_freq)
    iteration = 0
    while iteration < max_iterations:
        flash_times = poisson_flash_times(F, duration, duty_cycle)
        Fe = calculate_effective_frequency(flash_times, duration)
        if abs(Fe - target_Fe) <= tolerance:
            return min(F, max_freq), Fe, flash_times
        F = min(max(0.1, F + 0.15 if Fe < target_Fe else F - 0.15), max_freq)
        iteration += 1
    print(f"Warning: Max iterations reached for effective frequency {target_Fe} Hz.")
    return F, Fe, flash_times

def format_stp(time, frequency, duty_cycle, brightness, led_assignments):
    blocks = []
    for osc in range(4):
        led_str = ",".join(map(str, led_assignments[osc]))
        block = f"{time:.3f},1,{frequency:.2f},{frequency:.2f},{duty_cycle},{duty_cycle},{led_str},{brightness},{brightness}"
        blocks.append(block)
    return f'STP"{",".join(blocks)}"'

def get_led_assignments():
    led_config = []
    for osc in range(4):
        osc_leds = []
        print(f"Enter LED assignments for Oscillator {osc+1} (0 [off] or 1 [on]):")
        for led in range(4):
            value = int(input(f"  LED {led+1}: "))
            while value not in [0, 1]:
                print("  Please enter 0 or 1.")
                value = int(input(f"  LED {led+1}: "))
            osc_leds.append(value)
        led_config.append(osc_leds)
    return led_config

def generate_strobe_sequence():
    num_steps = int(input("Enter the number of steps in the sequence: "))
    output = []
    total_duration = 0

    step_params = []
    for step in range(num_steps):
        print(f"Step {step + 1}:")
        duration = float(input("  Enter step duration (seconds): "))
        start_Fe = float(input("  Enter start effective frequency (Hz): "))
        end_Fe = float(input("  Enter end effective frequency (Hz): "))
        start_l = int(input("  Enter start luminance (0-100): "))
        end_l = int(input("  Enter end luminance (0-100): "))
        start_d = int(input("  Enter start duty cycle (1-99): "))
        end_d = int(input("  Enter end duty cycle (1-99): "))
        wave_type = input("  Enter wave type (Square/Sine): ").strip()

        led_config = get_led_assignments()

        step_params.append((duration, start_Fe, end_Fe, start_l, end_l, start_d, end_d, wave_type, led_config))

    for step in step_params:
        duration, start_Fe, end_Fe, start_l, end_l, start_d, end_d, wave_type, led_config = step
        output.append(f'TIM"00:00:{int(total_duration // 60):02}:{total_duration % 60:04.1f}"')
        output.append(f'DUR"{duration:.1f}"')

        num_steps_estimated = int(duration * (start_Fe + end_Fe) / 2)
        min_interval = 0.025

        flash_times = [(i * (1.0 / start_Fe), (start_d / 100) * (1.0 / start_Fe)) for i in range(num_steps_estimated)]
        previous_onset = 0

        for onset, on_duration in flash_times:
            step_duration = max(onset - previous_onset, min_interval)
            previous_onset = onset + on_duration

            step_freq = start_Fe if onset < duration / 2 else end_Fe
            step_duty = start_d if onset < duration / 2 else end_d
            step_luminance = start_l if onset < duration / 2 else end_l

            stp_line = format_stp(step_duration, step_freq, step_duty, step_luminance, led_config)
            output.append(stp_line)

        total_duration += duration

    file_name = "strobe_sequence.txt"
    with open(file_name, "w") as f:
        f.write("\n".join(output))

    print(f"Generated {file_name}.")

generate_strobe_sequence()

Enter the number of steps in the sequence: 3
Step 1:
  Enter step duration (seconds): 10
  Enter start effective frequency (Hz): 10
  Enter end effective frequency (Hz): 10
  Enter start luminance (0-100): 50
  Enter end luminance (0-100): 50
  Enter start duty cycle (1-99): 50
  Enter end duty cycle (1-99): 50
  Enter wave type (Square/Sine): Square
Enter LED assignments for Oscillator 1 (0 [off] or 1 [on]):
  LED 1: 1
  LED 2: 1
  LED 3: 1
  LED 4: 1
Enter LED assignments for Oscillator 2 (0 [off] or 1 [on]):
  LED 1: 1
  LED 2: 1
  LED 3: 1
  LED 4: 1
Enter LED assignments for Oscillator 3 (0 [off] or 1 [on]):
  LED 1: 1
  LED 2: 1
  LED 3: 1
  LED 4: 1
Enter LED assignments for Oscillator 4 (0 [off] or 1 [on]):
  LED 1: 1
  LED 2: 1
  LED 3: 1
  LED 4: 1
Step 2:
  Enter step duration (seconds): 10
  Enter start effective frequency (Hz): 10
  Enter end effective frequency (Hz): 10
  Enter start luminance (0-100): 50
  Enter end luminance (0-100): 50
  Enter start duty cycle (1-99): 

4. After reviewing some of the parameter restrictions within the Session Manager, it seems that no step can be smaller than 0.2 seconds, which is perhaps why the device was unable to read any of our files. While this will certainly significantly impact our capacity for aperiodicity, the following script is a modification of the above (with precise LED modulation per each oscillator), while making sure that no parameter falls below 0.2 seconds.

In [2]:
import numpy as np

def poisson_flash_times(frequency, duration, duty_cycle, min_interval=0.200, max_interval_factor=3.0):
    cycle_length = 1.0 / frequency
    expected_flashes = int(duration * frequency)
    times = []
    time = 0
    while len(times) < expected_flashes and time < duration:
        interval = np.random.exponential(cycle_length * 0.8)
        interval = min(interval, cycle_length * max_interval_factor)
        interval = max(interval, min_interval)  # Ensure step >= 0.2s
        on_duration = interval * (duty_cycle / 100)
        time += interval
        if time < duration:
            times.append((time, on_duration))
    return times

def calculate_effective_frequency(flash_times, duration):
    return len(flash_times) / duration

def optimize_initial_frequency(target_Fe, duration, duty_cycle, tolerance=0.01, max_iterations=10000, max_freq=200.0):
    F = min(target_Fe * 1.2, max_freq)
    iteration = 0
    while iteration < max_iterations:
        flash_times = poisson_flash_times(F, duration, duty_cycle)
        Fe = calculate_effective_frequency(flash_times, duration)
        if abs(Fe - target_Fe) <= tolerance:
            return min(F, max_freq), Fe, flash_times
        F = min(max(0.1, F + 0.15 if Fe < target_Fe else F - 0.15), max_freq)
        iteration += 1
    print(f"Warning: Max iterations reached for effective frequency {target_Fe} Hz.")
    return F, Fe, flash_times

def format_stp(time, frequency, duty_cycle, brightness, led_assignments):
    blocks = []
    for osc in range(4):
        led_str = ",".join(map(str, led_assignments[osc]))
        block = f"{time:.3f},1,{frequency:.2f},{frequency:.2f},{duty_cycle},{duty_cycle},{led_str},{brightness},{brightness}"
        blocks.append(block)
    return f'STP"{",".join(blocks)}"'

def get_led_assignments():
    led_config = []
    for osc in range(4):
        osc_leds = []
        print(f"Enter LED assignments for Oscillator {osc+1} (0 [off] or 1 [on]):")
        for led in range(4):
            value = int(input(f"  LED {led+1}: "))
            while value not in [0, 1]:
                print("  Please enter 0 or 1.")
                value = int(input(f"  LED {led+1}: "))
            osc_leds.append(value)
        led_config.append(osc_leds)
    return led_config

def generate_strobe_sequence():
    stimulation = input("Choose stimulation type (Periodic/Aperiodic): ").strip()
    duration = float(input("Enter total sequence duration (seconds): "))
    start_Fe = float(input("Enter start effective frequency (Hz): "))
    end_Fe = float(input("Enter end effective frequency (Hz): "))
    start_l = int(input("Enter start luminance (0-100): "))
    end_l = int(input("Enter end luminance (0-100): "))
    start_d = int(input("Enter start duty cycle (1-99): "))
    end_d = int(input("Enter end duty cycle (1-99): "))
    wave_type = input("Enter wave type (Square/Sine): ").strip()

    led_config = get_led_assignments()

    output = [f'TIM"00:00:{int(duration // 60):02}:{duration % 60:04.1f}"', f'DUR"{duration:.1f}"']

    num_steps = int(duration * (start_Fe + end_Fe) / 2)

    if stimulation == "Aperiodic":
        F, Fe, flash_times = optimize_initial_frequency(start_Fe, duration, start_d)
    else:
        flash_times = [(i * (1.0 / start_Fe), (start_d / 100) * (1.0 / start_Fe)) for i in range(num_steps)]
        Fe = start_Fe

    previous_onset = 0
    step_lines = []

    for onset, on_duration in flash_times:
        step_duration = max(onset - previous_onset, 0.200)  # Enforce 0.2s minimum duration
        previous_onset = onset + on_duration

        step_freq = start_Fe if onset < duration / 2 else end_Fe
        step_duty = start_d if onset < duration / 2 else end_d
        step_luminance = start_l if onset < duration / 2 else end_l

        stp_line = format_stp(step_duration, step_freq, step_duty, step_luminance, led_config)
        step_lines.append(stp_line)

    output.extend(step_lines)

    file_name = "strobe_periodic.txt" if stimulation == "Periodic" else "strobe_aperiodic.txt"
    with open(file_name, "w") as f:
        f.write("\n".join(output))

    print(f"Generated {file_name}.")

generate_strobe_sequence()

Choose stimulation type (Periodic/Aperiodic): Periodic
Enter total sequence duration (seconds): 10
Enter start effective frequency (Hz): 10
Enter end effective frequency (Hz): 10
Enter start luminance (0-100): 50
Enter end luminance (0-100): 50
Enter start duty cycle (1-99): 50
Enter end duty cycle (1-99): 50
Enter wave type (Square/Sine): Square
Enter LED assignments for Oscillator 1 (0 [off] or 1 [on]):
  LED 1: 1
  LED 2: 1
  LED 3: 1
  LED 4: 1
Enter LED assignments for Oscillator 2 (0 [off] or 1 [on]):
  LED 1: 1
  LED 2: 1
  LED 3: 1
  LED 4: 1
Enter LED assignments for Oscillator 3 (0 [off] or 1 [on]):
  LED 1: 1
  LED 2: 1
  LED 3: 1
  LED 4: 1
Enter LED assignments for Oscillator 4 (0 [off] or 1 [on]):
  LED 1: 1
  LED 2: 1
  LED 3: 1
  LED 4: 1
Generated strobe_periodic.txt.


Well, that's not the case either -- and the explicit code chunk from the creator wasn't readable, either. Stay tuned for more!

5. This should format things so that the STP is not repeated 4x within each row, and is only present at the beginning of each STP row. Further, the DAY TIM is removed.

In [5]:
import numpy as np

def poisson_flash_times(frequency, duration, duty_cycle, min_interval=0.025, max_interval_factor=3.0):
    cycle_length = 1.0 / frequency
    expected_flashes = int(duration * frequency)
    times = []
    time = 0
    while len(times) < expected_flashes and time < duration:
        interval = np.random.exponential(cycle_length * 0.8)
        interval = min(interval, cycle_length * max_interval_factor)
        interval = max(interval, min_interval)
        on_duration = interval * (duty_cycle / 100)
        time += interval
        if time < duration:
            times.append((time, on_duration))
    return times

def calculate_effective_frequency(flash_times, duration):
    return len(flash_times) / duration

def optimize_initial_frequency(target_Fe, duration, duty_cycle, tolerance=0.01, max_iterations=10000, max_freq=200.0):
    F = min(target_Fe * 1.2, max_freq)
    iteration = 0
    while iteration < max_iterations:
        flash_times = poisson_flash_times(F, duration, duty_cycle)
        Fe = calculate_effective_frequency(flash_times, duration)
        if abs(Fe - target_Fe) <= tolerance:
            return min(F, max_freq), Fe, flash_times
        F = min(max(0.1, F + 0.15 if Fe < target_Fe else F - 0.15), max_freq)
        iteration += 1
    print(f"Warning: Max iterations reached for effective frequency {target_Fe} Hz.")
    return F, Fe, flash_times

def format_stp(time, frequency, duty_cycle, brightness, led_assignments):
    """
    Generates a single STP row where the time value appears only once.
    """
    blocks = []
    for osc in range(4):
        led_str = ",".join(map(str, led_assignments[osc]))
        block = f"1,{frequency:.2f},{frequency:.2f},{duty_cycle},{duty_cycle},{led_str},{brightness},{brightness}"
        blocks.append(block)
    return f'STP"{time:.3f},{",".join(blocks)}"'

def get_led_assignments():
    led_config = []
    for osc in range(4):
        osc_leds = []
        print(f"Enter LED assignments for Oscillator {osc+1} (0 [off] or 1 [on]):")
        for led in range(4):
            value = int(input(f"  LED {led+1}: "))
            while value not in [0, 1]:
                print("  Please enter 0 or 1.")
                value = int(input(f"  LED {led+1}: "))
            osc_leds.append(value)
        led_config.append(osc_leds)
    return led_config

def generate_strobe_sequence():
    num_steps = int(input("Enter the number of steps in the sequence: "))
    output = []
    total_duration = 0

    step_params = []
    for step in range(num_steps):
        print(f"Step {step + 1}:")
        duration = float(input("  Enter step duration (seconds): "))
        start_Fe = float(input("  Enter start effective frequency (Hz): "))
        end_Fe = float(input("  Enter end effective frequency (Hz): "))
        start_l = int(input("  Enter start luminance (0-100): "))
        end_l = int(input("  Enter end luminance (0-100): "))
        start_d = int(input("  Enter start duty cycle (1-99): "))
        end_d = int(input("  Enter end duty cycle (1-99): "))
        wave_type = input("  Enter wave type (Square/Sine): ").strip()

        led_config = get_led_assignments()

        step_params.append((duration, start_Fe, end_Fe, start_l, end_l, start_d, end_d, wave_type, led_config))

    for step in step_params:
        duration, start_Fe, end_Fe, start_l, end_l, start_d, end_d, wave_type, led_config = step
        output.append(f'TIM"00:{int(total_duration // 60):02}:{total_duration % 60:04.1f}"')
        output.append(f'DUR"{duration:.1f}"')

        num_steps_estimated = int(duration * (start_Fe + end_Fe) / 2)
        min_interval = 0.025

        flash_times = [(i * (1.0 / start_Fe), (start_d / 100) * (1.0 / start_Fe)) for i in range(num_steps_estimated)]
        previous_onset = 0

        for onset, on_duration in flash_times:
            step_duration = max(onset - previous_onset, min_interval)
            previous_onset = onset + on_duration

            step_freq = start_Fe if onset < duration / 2 else end_Fe
            step_duty = start_d if onset < duration / 2 else end_d
            step_luminance = start_l if onset < duration / 2 else end_l

            stp_line = format_stp(step_duration, step_freq, step_duty, step_luminance, led_config)
            output.append(stp_line)

        total_duration += duration

    file_name = "strobe_sequence.txt"
    with open(file_name, "w") as f:
        f.write("\n".join(output))

    print(f"Generated {file_name}.")

generate_strobe_sequence()

Enter the number of steps in the sequence: 3
Step 1:
  Enter step duration (seconds): 10
  Enter start effective frequency (Hz): 10
  Enter end effective frequency (Hz): 10
  Enter start luminance (0-100): 50
  Enter end luminance (0-100): 50
  Enter start duty cycle (1-99): 50
  Enter end duty cycle (1-99): 50
  Enter wave type (Square/Sine): Square
Enter LED assignments for Oscillator 1 (0 [off] or 1 [on]):
  LED 1: 1
  LED 2: 1
  LED 3: 1
  LED 4: 1
Enter LED assignments for Oscillator 2 (0 [off] or 1 [on]):
  LED 1: 1
  LED 2: 1
  LED 3: 1
  LED 4: 1
Enter LED assignments for Oscillator 3 (0 [off] or 1 [on]):
  LED 1: 1
  LED 2: 1
  LED 3: 1
  LED 4: 1
Enter LED assignments for Oscillator 4 (0 [off] or 1 [on]):
  LED 1: 1
  LED 2: 1
  LED 3: 1
  LED 4: 1
Step 2:
  Enter step duration (seconds): 10
  Enter start effective frequency (Hz): 10
  Enter end effective frequency (Hz): 10
  Enter start luminance (0-100): 50
  Enter end luminance (0-100): 50
  Enter start duty cycle (1-99): 

6. This is the same code as above, with 200ms minimum step durations. To change the minimum accepted value, simply change the value within the poisson_flash_times function for min_interval = x. Further, the DAY TIM is removed.

In [6]:
import numpy as np

def poisson_flash_times(frequency, duration, duty_cycle, min_interval=0.200, max_interval_factor=3.0):
    cycle_length = 1.0 / frequency
    expected_flashes = int(duration * frequency)
    times = []
    time = 0
    while len(times) < expected_flashes and time < duration:
        interval = np.random.exponential(cycle_length * 0.8)
        interval = min(interval, cycle_length * max_interval_factor)
        interval = max(interval, min_interval)  # Ensure step >= 0.2s
        on_duration = interval * (duty_cycle / 100)
        time += interval
        if time < duration:
            times.append((time, on_duration))
    return times

def calculate_effective_frequency(flash_times, duration):
    return len(flash_times) / duration

def optimize_initial_frequency(target_Fe, duration, duty_cycle, tolerance=0.01, max_iterations=10000, max_freq=200.0):
    F = min(target_Fe * 1.2, max_freq)
    iteration = 0
    while iteration < max_iterations:
        flash_times = poisson_flash_times(F, duration, duty_cycle)
        Fe = calculate_effective_frequency(flash_times, duration)
        if abs(Fe - target_Fe) <= tolerance:
            return min(F, max_freq), Fe, flash_times
        F = min(max(0.1, F + 0.15 if Fe < target_Fe else F - 0.15), max_freq)
        iteration += 1
    print(f"Warning: Max iterations reached for effective frequency {target_Fe} Hz.")
    return F, Fe, flash_times

def format_stp(time, frequency, duty_cycle, brightness, led_assignments):
    """
    Generates a single STP row where the time value appears only once.
    """
    blocks = []
    for osc in range(4):
        led_str = ",".join(map(str, led_assignments[osc]))
        block = f"1,{frequency:.2f},{frequency:.2f},{duty_cycle},{duty_cycle},{led_str},{brightness},{brightness}"
        blocks.append(block)
    return f'STP"{time:.3f},{",".join(blocks)}"'

def get_led_assignments():
    led_config = []
    for osc in range(4):
        osc_leds = []
        print(f"Enter LED assignments for Oscillator {osc+1} (0 [off] or 1 [on]):")
        for led in range(4):
            value = int(input(f"  LED {led+1}: "))
            while value not in [0, 1]:
                print("  Please enter 0 or 1.")
                value = int(input(f"  LED {led+1}: "))
            osc_leds.append(value)
        led_config.append(osc_leds)
    return led_config

def generate_strobe_sequence():
    stimulation = input("Choose stimulation type (Periodic/Aperiodic): ").strip()
    num_steps = int(input("Enter the number of steps in the sequence: "))
    output = []
    total_duration = 0

    step_params = []
    for step in range(num_steps):
        print(f"Step {step + 1}:")
        duration = float(input("  Enter step duration (seconds): "))
        start_Fe = float(input("  Enter start effective frequency (Hz): "))
        end_Fe = float(input("  Enter end effective frequency (Hz): "))
        start_l = int(input("  Enter start luminance (0-100): "))
        end_l = int(input("  Enter end luminance (0-100): "))
        start_d = int(input("  Enter start duty cycle (1-99): "))
        end_d = int(input("  Enter end duty cycle (1-99): "))
        wave_type = input("  Enter wave type (Square/Sine): ").strip()

        led_config = get_led_assignments()

        step_params.append((duration, start_Fe, end_Fe, start_l, end_l, start_d, end_d, wave_type, led_config))

    for step in step_params:
        duration, start_Fe, end_Fe, start_l, end_l, start_d, end_d, wave_type, led_config = step
        output.append(f'TIM"00:{int(total_duration // 60):02}:{total_duration % 60:04.1f}"')
        output.append(f'DUR"{duration:.1f}"')

        if stimulation == "Aperiodic":
            F, Fe, flash_times = optimize_initial_frequency(start_Fe, duration, start_d)
        else:
            num_steps_estimated = int(duration * (start_Fe + end_Fe) / 2)
            flash_times = [(i * (1.0 / start_Fe), (start_d / 100) * (1.0 / start_Fe)) for i in range(num_steps_estimated)]

        previous_onset = 0
        step_lines = []

        for onset, on_duration in flash_times:
            step_duration = max(onset - previous_onset, 0.200)  # Enforce 0.2s minimum duration
            previous_onset = onset + on_duration

            step_freq = start_Fe if onset < duration / 2 else end_Fe
            step_duty = start_d if onset < duration / 2 else end_d
            step_luminance = start_l if onset < duration / 2 else end_l

            stp_line = format_stp(step_duration, step_freq, step_duty, step_luminance, led_config)
            step_lines.append(stp_line)

        output.extend(step_lines)
        total_duration += duration

    file_name = "strobe_periodic.txt" if stimulation == "Periodic" else "strobe_aperiodic.txt"
    with open(file_name, "w") as f:
        f.write("\n".join(output))

    print(f"Generated {file_name}.")

generate_strobe_sequence()

Choose stimulation type (Periodic/Aperiodic): Periodic
Enter the number of steps in the sequence: 1
Step 1:
  Enter step duration (seconds): 10
  Enter start effective frequency (Hz): 10
  Enter end effective frequency (Hz): 10
  Enter start luminance (0-100): 50
  Enter end luminance (0-100): 50
  Enter start duty cycle (1-99): 50
  Enter end duty cycle (1-99): 50
  Enter wave type (Square/Sine): Square
Enter LED assignments for Oscillator 1 (0 [off] or 1 [on]):
  LED 1: 1
  LED 2: 1
  LED 3: 1
  LED 4: 1
Enter LED assignments for Oscillator 2 (0 [off] or 1 [on]):
  LED 1: 1
  LED 2: 1
  LED 3: 1
  LED 4: 1
Enter LED assignments for Oscillator 3 (0 [off] or 1 [on]):
  LED 1: 1
  LED 2: 1
  LED 3: 1
  LED 4: 1
Enter LED assignments for Oscillator 4 (0 [off] or 1 [on]):
  LED 1: 1
  LED 2: 1
  LED 3: 1
  LED 4: 1
Generated strobe_periodic.txt.


7. After reviewing a working session from Jimi, it looks like the final change we need to add is how the STP is adjusted incremently. Here is an adjustment of #5, with the very granular timestamps that may not work on the device. An additional adjustment has been made to correctly format the TIM after a maladaption re: TIM.

In [7]:
import numpy as np

def poisson_flash_times(frequency, duration, duty_cycle, min_interval=0.025, max_interval_factor=3.0):
    cycle_length = 1.0 / frequency
    expected_flashes = int(duration * frequency)
    times = []
    time = 0
    while len(times) < expected_flashes and time < duration:
        interval = np.random.exponential(cycle_length * 0.8)
        interval = min(interval, cycle_length * max_interval_factor)
        interval = max(interval, min_interval)
        on_duration = interval * (duty_cycle / 100)
        time += interval
        if time < duration:
            times.append((time, on_duration))
    return times

def calculate_effective_frequency(flash_times, duration):
    return len(flash_times) / duration

def optimize_initial_frequency(target_Fe, duration, duty_cycle, tolerance=0.01, max_iterations=10000, max_freq=200.0):
    F = min(target_Fe * 1.2, max_freq)
    iteration = 0
    while iteration < max_iterations:
        flash_times = poisson_flash_times(F, duration, duty_cycle)
        Fe = calculate_effective_frequency(flash_times, duration)
        if abs(Fe - target_Fe) <= tolerance:
            return min(F, max_freq), Fe, flash_times
        F = min(max(0.1, F + 0.15 if Fe < target_Fe else F - 0.15), max_freq)
        iteration += 1
    print(f"Warning: Max iterations reached for effective frequency {target_Fe} Hz.")
    return F, Fe, flash_times

def format_stp(time, frequency, duty_cycle, brightness, led_assignments, stp_index):
    """
    Generates a single STP row where the time value appears only once.
    """
    blocks = []
    for osc in range(4):
        led_str = ",".join(map(str, led_assignments[osc]))
        block = f"{stp_index},{frequency:.2f},{frequency:.2f},{duty_cycle},{duty_cycle},{led_str},{brightness},{brightness}"
        blocks.append(block)
    return f'STP"{time:.3f},{",".join(blocks)}"'

def get_led_assignments():
    led_config = []
    for osc in range(4):
        osc_leds = []
        print(f"Enter LED assignments for Oscillator {osc+1} (0 [off] or 1 [on]):")
        for led in range(4):
            value = int(input(f"  LED {led+1}: "))
            while value not in [0, 1]:
                print("  Please enter 0 or 1.")
                value = int(input(f"  LED {led+1}: "))
            osc_leds.append(value)
        led_config.append(osc_leds)
    return led_config

def generate_strobe_sequence():
    num_steps = int(input("Enter the number of steps in the sequence: "))
    output = []
    total_duration = 0
    stp_index = 1  # Initialize STP index

    step_params = []
    for step in range(num_steps):
        print(f"Step {step + 1}:")
        duration = float(input("  Enter step duration (seconds): "))
        start_Fe = float(input("  Enter start effective frequency (Hz): "))
        end_Fe = float(input("  Enter end effective frequency (Hz): "))
        start_l = int(input("  Enter start luminance (0-100): "))
        end_l = int(input("  Enter end luminance (0-100): "))
        start_d = int(input("  Enter start duty cycle (1-99): "))
        end_d = int(input("  Enter end duty cycle (1-99): "))
        wave_type = input("  Enter wave type (Square/Sine): ").strip()

        led_config = get_led_assignments()

        step_params.append((duration, start_Fe, end_Fe, start_l, end_l, start_d, end_d, wave_type, led_config))

    for step in step_params:
        duration, start_Fe, end_Fe, start_l, end_l, start_d, end_d, wave_type, led_config = step
        output.append(f'TIM\"00:{int((total_duration + duration) // 60):02}:{(total_duration + duration) % 60:04.1f}\"')
        output.append(f'DUR"{duration:.1f}"')

        num_steps_estimated = int(duration * (start_Fe + end_Fe) / 2)
        min_interval = 0.025

        flash_times = [(i * (1.0 / start_Fe), (start_d / 100) * (1.0 / start_Fe)) for i in range(num_steps_estimated)]
        previous_onset = 0

        for onset, on_duration in flash_times:
            step_duration = max(onset - previous_onset, min_interval)
            previous_onset = onset + on_duration

            step_freq = start_Fe if onset < duration / 2 else end_Fe
            step_duty = start_d if onset < duration / 2 else end_d
            step_luminance = start_l if onset < duration / 2 else end_l

            stp_line = format_stp(step_duration, step_freq, step_duty, step_luminance, led_config, stp_index)
            output.append(stp_line)

            stp_index += 1  # Increment STP index for next row

        total_duration += duration

    file_name = "strobe_sequence.txt"
    with open(file_name, "w") as f:
        f.write("\n".join(output))

    print(f"Generated {file_name}.")

generate_strobe_sequence()

Enter the number of steps in the sequence: 1
Step 1:
  Enter step duration (seconds): 10
  Enter start effective frequency (Hz): 10
  Enter end effective frequency (Hz): 10
  Enter start luminance (0-100): 50
  Enter end luminance (0-100): 50
  Enter start duty cycle (1-99): 50
  Enter end duty cycle (1-99): 50
  Enter wave type (Square/Sine): Square
Enter LED assignments for Oscillator 1 (0 [off] or 1 [on]):
  LED 1: 1
  LED 2: 1
  LED 3: 1
  LED 4: 1
Enter LED assignments for Oscillator 2 (0 [off] or 1 [on]):
  LED 1: 1
  LED 2: 1
  LED 3: 1
  LED 4: 1
Enter LED assignments for Oscillator 3 (0 [off] or 1 [on]):
  LED 1: 1
  LED 2: 1
  LED 3: 1
  LED 4: 1
Enter LED assignments for Oscillator 4 (0 [off] or 1 [on]):
  LED 1: 1
  LED 2: 1
  LED 3: 1
  LED 4: 1
Generated strobe_sequence.txt.


7. An adjustment of #6, with correct incremental STP adjustments. TIM has also been adjusted per the maladjustment specified above.

In [8]:
import numpy as np

def poisson_flash_times(frequency, duration, duty_cycle, min_interval=0.200, max_interval_factor=3.0):
    cycle_length = 1.0 / frequency
    expected_flashes = int(duration * frequency)
    times = []
    time = 0
    while len(times) < expected_flashes and time < duration:
        interval = np.random.exponential(cycle_length * 0.8)
        interval = min(interval, cycle_length * max_interval_factor)
        interval = max(interval, min_interval)  # Ensure step >= 0.2s
        on_duration = interval * (duty_cycle / 100)
        time += interval
        if time < duration:
            times.append((time, on_duration))
    return times

def calculate_effective_frequency(flash_times, duration):
    return len(flash_times) / duration

def optimize_initial_frequency(target_Fe, duration, duty_cycle, tolerance=0.01, max_iterations=10000, max_freq=200.0):
    F = min(target_Fe * 1.2, max_freq)
    iteration = 0
    while iteration < max_iterations:
        flash_times = poisson_flash_times(F, duration, duty_cycle)
        Fe = calculate_effective_frequency(flash_times, duration)
        if abs(Fe - target_Fe) <= tolerance:
            return min(F, max_freq), Fe, flash_times
        F = min(max(0.1, F + 0.15 if Fe < target_Fe else F - 0.15), max_freq)
        iteration += 1
    print(f"Warning: Max iterations reached for effective frequency {target_Fe} Hz.")
    return F, Fe, flash_times

def format_stp(time, frequency, duty_cycle, brightness, led_assignments):
    """
    Generates a single STP row where the time value appears only once.
    """
    blocks = []
    for osc in range(4):
        led_str = ",".join(map(str, led_assignments[osc]))
        block = f"1,{frequency:.2f},{frequency:.2f},{duty_cycle},{duty_cycle},{led_str},{brightness},{brightness}"
        blocks.append(block)
    return f'STP"{time:.3f},{",".join(blocks)}"'

def get_led_assignments():
    led_config = []
    for osc in range(4):
        osc_leds = []
        print(f"Enter LED assignments for Oscillator {osc+1} (0 [off] or 1 [on]):")
        for led in range(4):
            value = int(input(f"  LED {led+1}: "))
            while value not in [0, 1]:
                print("  Please enter 0 or 1.")
                value = int(input(f"  LED {led+1}: "))
            osc_leds.append(value)
        led_config.append(osc_leds)
    return led_config

def generate_strobe_sequence():
    stimulation = input("Choose stimulation type (Periodic/Aperiodic): ").strip()
    num_steps = int(input("Enter the number of steps in the sequence: "))
    output = []
    total_duration = 0

    step_params = []
    for step in range(num_steps):
        print(f"Step {step + 1}:")
        duration = float(input("  Enter step duration (seconds): "))
        start_Fe = float(input("  Enter start effective frequency (Hz): "))
        end_Fe = float(input("  Enter end effective frequency (Hz): "))
        start_l = int(input("  Enter start luminance (0-100): "))
        end_l = int(input("  Enter end luminance (0-100): "))
        start_d = int(input("  Enter start duty cycle (1-99): "))
        end_d = int(input("  Enter end duty cycle (1-99): "))
        wave_type = input("  Enter wave type (Square/Sine): ").strip()

        led_config = get_led_assignments()

        step_params.append((duration, start_Fe, end_Fe, start_l, end_l, start_d, end_d, wave_type, led_config))

    for step in step_params:
        duration, start_Fe, end_Fe, start_l, end_l, start_d, end_d, wave_type, led_config = step
        output.append(f'TIM\"00:{int((total_duration + duration) // 60):02}:{(total_duration + duration) % 60:04.1f}\"')
        output.append(f'DUR"{duration:.1f}"')

        if stimulation == "Aperiodic":
            F, Fe, flash_times = optimize_initial_frequency(start_Fe, duration, start_d)
        else:
            num_steps_estimated = int(duration * (start_Fe + end_Fe) / 2)
            flash_times = [(i * (1.0 / start_Fe), (start_d / 100) * (1.0 / start_Fe)) for i in range(num_steps_estimated)]

        previous_onset = 0
        step_lines = []

        for onset, on_duration in flash_times:
            step_duration = max(onset - previous_onset, 0.200)  # Enforce 0.2s minimum duration
            previous_onset = onset + on_duration

            step_freq = start_Fe if onset < duration / 2 else end_Fe
            step_duty = start_d if onset < duration / 2 else end_d
            step_luminance = start_l if onset < duration / 2 else end_l

            stp_line = format_stp(step_duration, step_freq, step_duty, step_luminance, led_config)
            step_lines.append(stp_line)

        output.extend(step_lines)
        total_duration += duration

    file_name = "strobe_periodic.txt" if stimulation == "Periodic" else "strobe_aperiodic.txt"
    with open(file_name, "w") as f:
        f.write("\n".join(output))

    print(f"Generated {file_name}.")

generate_strobe_sequence()

Choose stimulation type (Periodic/Aperiodic): Periodic
Enter the number of steps in the sequence: 1
Step 1:
  Enter step duration (seconds): 10
  Enter start effective frequency (Hz): 10
  Enter end effective frequency (Hz): 10
  Enter start luminance (0-100): 50
  Enter end luminance (0-100): 50
  Enter start duty cycle (1-99): 50
  Enter end duty cycle (1-99): 50
  Enter wave type (Square/Sine): Square
Enter LED assignments for Oscillator 1 (0 [off] or 1 [on]):
  LED 1: 1
  LED 2: 1
  LED 3: 1
  LED 4: 1
Enter LED assignments for Oscillator 2 (0 [off] or 1 [on]):
  LED 1: 1
  LED 2: 1
  LED 3: 1
  LED 4: 1
Enter LED assignments for Oscillator 3 (0 [off] or 1 [on]):
  LED 1: 1
  LED 2: 1
  LED 3: 1
  LED 4: 1
Enter LED assignments for Oscillator 4 (0 [off] or 1 [on]):
  LED 1: 1
  LED 2: 1
  LED 3: 1
  LED 4: 1
Generated strobe_periodic.txt.
